## This notebook launches the LatissCWFSAlign script, meant for the scriptQueue, but via a notebook.
##### This calculates focus offsets for the hexapod (and also decentering corrections)

#### WARNING: This currently requires ts_externalscripts branch **tickets/DM-28423**

In [1]:
import sys
import asyncio
import time

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign

from lsst.ts.idl.enums.Script import ScriptState

/home/patrickingraham/repos/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_acquire_and_take_sequence.py:37: UserWarning: Cannot import lsst.observing. Script will not work.
  warnings.warn("Cannot import lsst.observing. Script will not work.")
/home/patrickingraham/repos/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_acquire_target.py:41: UserWarning: Cannot import lsst.observing. Script will not work.
  warnings.warn("Cannot import lsst.observing. Script will not work.")
/home/patrickingraham/repos/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_take_sequence.py:36: UserWarning: Cannot import lsst.observing. Script will not work.
  warnings.warn("Cannot import lsst.observing. Script will not work.")


In [4]:
# for autocompleted to work
%config IPCompleter.use_jedi = False

logMessage DDS read queue is filling: 40 of 100 elements
logMessage DDS read queue is filling: 59 of 100 elements
logMessage DDS read queue is filling: 52 of 100 elements
logMessage DDS read queue is filling: 57 of 100 elements
logMessage DDS read queue is filling: 10 of 100 elements
logMessage DDS read queue is filling: 10 of 100 elements
logMessage DDS read queue is filling: 10 of 100 elements
logMessage DDS read queue is filling: 14 of 100 elements


In [2]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [3]:
script = LatissCWFSAlign(index=1, remotes=True)  # this essentially calls the init method
# make sure all remotes etc are running
await script.start_task

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.06 sec
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, algorithm)
Read 9 history items for RemoteEvent(ATDomeTrajectory, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, authList)
Read 100 history items for RemoteEvent(ATDomeTrajectory, 0, heartbeat)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, logLevel)
Read 96 history items for RemoteEvent(ATDomeTrajectory, 0, logMessage)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingVersions)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingsApplied)
Read 1 history i

In [ ]:
# set wrap strategy
# this is required until the ATPtg is updated to not configure the mount for maximum time on target
# script.atcs.rem.atptg.cmd_raDecTarget.set(azWrapStrategy=1)  # 1 does not unwrap, 0 unwraps

## Emulate how the scriptQueue launches scripts
##### Start here if re-running the script after a correction

In [ ]:
configuration = yaml.safe_dump({"filter": 'empty_1', 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook'})
print(configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# Target must be on the detector
# Can use this command to slew to target if required
# await script.attcs.slew_object('96 Cet')

In [ ]:
# Run this script while taking new data
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10
results = await script.run_cwfs()

In [ ]:
# show donuts and centroids
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title(f"intra visitID - {script.intra_visit_id}")
ax11.imshow(script.I1[0].image0)
ax11.contour(script.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title(f"extra visitID - {script.extra_visit_id}")
ax12.imshow(script.I2[0].image0)
ax12.contour(script.algo.pMask) 

In [ ]:
# Apply calculated focus offset
calculated_hexapod_focus_offset = results['hex_offset'][2]
print(f'Applying focus offset of {calculated_hexapod_focus_offset}')
await script.atcs.rem.ataos.cmd_offset.set_start(z=calculated_hexapod_focus_offset)

# Stop here unless a re-reduction of the doughnuts is required.

#### If you want to re-reduce data then use the below cells

In [ ]:
# Show which files/parameters were taken in the sequence above
print(f'intra_visit_id is {script.intra_visit_id}')
print(f'extra_visit_id is {script.extra_visit_id}')
print(f'angle is {script.angle}')

In [ ]:
# If desired then different filenames can be manually input here
#script.intra_visit_id=2021011900169 
#script.extra_visit_id=2021011900170 
#script.angle=-91.56748047249727

In [ ]:
# reruns reduction part only. ALL 3 fields above must be set! 
rerun_results = await script.run_cwfs()